#                              Capstone Course Final Assignment
   ## Gilad Goldenberg
   ## 27/8/2020 
   
   
   # Best Culinary Neighbourood 

## Importing libraries for the work:

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
#from pandas.io.json import pd.json_normalize
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Folium installed
Libraries imported.


### Define Foursquare Credentials and Version

In [3]:
CLIENT_ID = 'IGH2TO1PRJLFXGGDC4F3K4X2K0Q0PE2LBO4KK0JQHOHVY5XK' # your Foursquare ID
CLIENT_SECRET = 'XRYG2XUCFSZJ1YJNQNSWADQGUQ4JYITBCIW1HPXCYP5YK5KL' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IGH2TO1PRJLFXGGDC4F3K4X2K0Q0PE2LBO4KK0JQHOHVY5XK
CLIENT_SECRET:XRYG2XUCFSZJ1YJNQNSWADQGUQ4JYITBCIW1HPXCYP5YK5KL


## Geting NY Neighbourhood Data

In [4]:
#Getting all neghibourhoods in NY:
import json # library to handle JSON files

!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
newyork_data


Data downloaded!


{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [5]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
...,...,...,...,...
301,Manhattan,Hudson Yards,40.756658,-74.000111
302,Queens,Hammels,40.587338,-73.805530
303,Queens,Bayswater,40.611322,-73.765968
304,Queens,Queensbridge,40.756091,-73.945631


In [6]:
search_query = 'Restaurants'
radius = 500
print(search_query + ' .... OK!')

Restaurants .... OK!


In [7]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [8]:
neighborhoods.Neighborhood.iloc[0]

'Wakefield'

In [9]:
# New ID
CLIENT_ID = '2ZZYFD4ZG35T3DTCPJVK4J4ZYVHBQAOQWP5J3GXCRWPNO4HD' # your Foursquare ID
CLIENT_SECRET = 'BM2WU3GY43DDZGPYP441ZCMZIHODLAEC1AGJI4NORZYJ3XWG' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2ZZYFD4ZG35T3DTCPJVK4J4ZYVHBQAOQWP5J3GXCRWPNO4HD
CLIENT_SECRET:BM2WU3GY43DDZGPYP441ZCMZIHODLAEC1AGJI4NORZYJ3XWG


#### Getting venues dictionaries from each neighbourhood, if there are no venues in the neighbourhood, skip it:

In [10]:
dataframe={}
j=0
compArray=np.arange(len(neighborhoods.index))
for i in neighborhoods.index:
    latitude = neighborhoods.Latitude.iloc[i]
    longitude =  neighborhoods.Longitude.iloc[i]    
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json() # get results in a complicated dictionary ...
    try: 
        venues = results['response']['venues']# # assign relevant part of JSON to venues
    except:
        g=1
    if(venues):
        #dataframe[j]['neighbourhood_name']=0
        dataframe[j]  = pd.json_normalize(venues) # tranform venues into a dataframe       
        dataframe[j].loc[:,'neighbourhood_name'] = neighborhoods.Neighborhood.iloc[i]
        compArray[j]=i
        j=j+1
dataframe

{0:                          id                                   name  \
 0  4f21c904e4b0831873dc6256  Tony & Cyndi's Pizzeria & Restaurants   
 1  59e644cc66f3cd17e0eeefac                                 Subway   
 
                                           categories    referralId  hasPerk  \
 0  [{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...  v-1598510407    False   
 1  [{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...  v-1598510407    False   
 
   location.address location.crossStreet  location.lat  location.lng  \
 0             5761             Broadway     40.883566    -73.901809   
 1   193 W 237th St             Broadway     40.884486    -73.900275   
 
                              location.labeledLatLngs  location.distance  \
 0  [{'label': 'display', 'lat': 40.88356615627457...                225   
 1  [{'label': 'display', 'lat': 40.88448638079450...                377   
 
   location.postalCode location.cc location.city location.state  \
 0               10463   

### Create a convinient dataframe of venues for each neighbourhood.

In [12]:
dataframe_filtered = {}
for i in dataframe:
# keep only columns that include venue name, and anything that is associated with location
    filtered_columns = ['name', 'categories'] + [col for col in dataframe[i].columns if col.startswith('location.')] + ['id'] +['neighbourhood_name']
    dataframe_filtered[i] = dataframe[i].loc[:, filtered_columns]
    # filter the category for each row
    dataframe_filtered[i]['categories'] = dataframe_filtered[i].apply(get_category_type, axis=1)
    # clean column names by keeping only last term
    dataframe_filtered[i].columns = [column.split('.')[-1] for column in dataframe_filtered[i].columns]
dataframe_filtered
datasave=dataframe_filtered

### Check for venues with ratings:

In [14]:

for i in dataframe_filtered:# dataframe_filtered.index: 
    for j in dataframe_filtered[i].index:
        dataframe_filtered[i]['rating'] =0
        venue_id = dataframe_filtered[i].id.iloc[j] # ID of a resturant
        url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
        result = requests.get(url).json()
    try:
        dataframe_filtered[i]['rating'].iloc[j] = result['response']['venue']['rating']
    except:
        dataframe_filtered[i]['rating'].iloc[j] = -1
    print(".")
        
dataframe_filtered[0]


C:\Users\giladgo\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id,neighbourhood_name,rating
0,Tony & Cyndi's Pizzeria & Restaurants,Pizza Place,5761,Broadway,40.883566,-73.901809,"[{'label': 'display', 'lat': 40.88356615627457...",225,10463,US,Bronx,NY,United States,"[5761 (Broadway), Bronx, NY 10463, United States]",4f21c904e4b0831873dc6256,Kingsbridge,0.0
1,Subway,Sandwich Place,193 W 237th St,Broadway,40.884486,-73.900275,"[{'label': 'display', 'lat': 40.88448638079450...",377,10463,US,Bronx,NY,United States,"[193 W 237th St (Broadway), Bronx, NY 10463, U...",59e644cc66f3cd17e0eeefac,Kingsbridge,6.6


In [15]:
k=0
g=0
l=0
ratingArray=np.arange(len(dataframe_filtered))

BestRatingNeighNumber = -1
BestRatingNeigh = -1
for i in dataframe_filtered:
    for j in dataframe_filtered[i].index:
        if (dataframe_filtered[i].rating[j] > -1):
            k=k+1
    if (dataframe_filtered[i].rating.mean() > BestRatingNeigh):
        BestRatingNeigh = dataframe_filtered[i].rating.mean()
        BestRatingNeighNumber = i


for i in dataframe_filtered:
    for j in dataframe_filtered[i].index: 
        if (dataframe_filtered[i].rating[j]>-1):
            l=l+1
    ratingArray[i] = l
    l=0
    
#print(k)
#print(BestRatingNeigh)
#print(BestRatingNeighNumber)
print('Best culinary place is: ' + str(dataframe_filtered[BestRatingNeighNumber].neighbourhood_name[0]) +' With: ' 
      +str(ratingArray[BestRatingNeighNumber]) + ' Ratings and an average rating of: ' 
      + str(dataframe_filtered[BestRatingNeighNumber].rating.mean()) + ' Points !')
print('Neighbourhood with the most rated resturants is: ' + str(dataframe_filtered[np.argmax(ratingArray)].neighbourhood_name[0]) 
      +' With: ' + str(ratingArray.max()) + ' Ratings  and an average of: ' 
      +str(dataframe_filtered[np.argmax(ratingArray)].rating.mean()) + ' Points !')
print('Best Culinary neighbourhod:')


Best culinary place is: Upper East Side With: 1 Ratings and an average rating of: 8.0 Points !
Neighbourhood with the most rated resturants is: Chinatown With: 7 Ratings  and an average of: 1.0571428571428572 Points !
Best Culinary neighbourhod:


#### Let's visualize the restaurants  that where picked !

In [18]:
venues_map = folium.Map(location=[dataframe_filtered[BestRatingNeighNumber].lat[0],
                                  dataframe_filtered[BestRatingNeighNumber].lng[0]], zoom_start=12) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [dataframe_filtered[BestRatingNeighNumber].lat[0],
                                  dataframe_filtered[BestRatingNeighNumber].lng[0]],
    radius=10,
    color='red',
    popup='Best Rated Neighborhood: ' +str(dataframe_filtered[BestRatingNeighNumber].neighbourhood_name[0]),
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers

folium.features.CircleMarker(
    [dataframe_filtered[np.argmax(ratingArray)].lat[0], dataframe_filtered[np.argmax(ratingArray)].lng[0]],
    radius=5,
    color='blue',
    popup='Most Rated Neighborhood: ' +str(dataframe_filtered[np.argmax(ratingArray)].neighbourhood_name[0]),
    fill = True,
    fill_color='blue',
    fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map